In [2]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os
from PIL import Image

#Function takes as input the path to nii.gz file, and the maximum distance of the catheter inside the patient (in mm)

def slice_ni(path, max_cath_dist):
    
    # Load a particular nii.gz file
    stacked_img = nib.load(path) 
    
    # Convert Image object to numpy array 
    stacked_array = np.array(stacked_img.dataobj) 

    # Specify dimensions
    width = np.size(stacked_array, axis = 0)
    height = np.size(stacked_array, axis = 1)
    depth = np.size(stacked_array, axis =2)
    channels = 3
    
    # Catheter position ranges from 1mm to 55mm and we assume constant pullback speed
    points = [(1,1),(depth, max_cath_dist)] 
    x_coords, y_coords = zip(*points)
    A = np.vstack([x_coords,np.ones(len(x_coords))]).T
    m, c = np.linalg.lstsq(A, y_coords)[0]

    for i in range(depth):
    
        slice_array = stacked_array[:,:,i]

        # Create an empty image
        img = np.zeros((height, width, channels), dtype=np.uint8)

        # Set the RGB values (array of 3-tuples must be converted to an array of depth 3, one for each channel)
        for y in range(img.shape[0]):
            for x in range(img.shape[1]):
            
                img[y][x][0] = slice_array[x,y][0] #R
                img[y][x][1] = slice_array[x,y][1] #G
                img[y][x][2] = slice_array[x,y][2] #B
        
        # Correct image orientation
        img = np.flipud(img)
        img = np.fliplr(img)
        
        # Naming format of .png images are *_(slice number)_(position in mm).png
        im = Image.fromarray(img)
        im.save('./files/image_' + str(i+1) + '_' + str(np.round((m*(i+1) + c), 2)) + '.png')